In [ ]:
# Code sử dụng cho môi trường kaggle
!conda install -y gdown

In [ ]:
#Code sử dụng cho môi trường kaggle
!gdown --id 185QQjlWWs5Ol7vzRW6ioMbFg08Bp3kVW
!gdown --id 1U1Qh8k3oscv840Kop3hyNJJRHjzSWMS0

In [ ]:
# Load Drive Helper
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

## Nạp và Tiền xử lý dữ liệu

In [ ]:
# Import các thư viên cần thiết
import os
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.metrics import (precision_score, recall_score, f1_score, accuracy_score)

import tensorflow as tf

2024-06-08 15:05:45.654794: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-08 15:05:45.654847: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-08 15:05:45.656436: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# Đường dẫn dành cho google colab
# DATASET_PATH = '/content/drive/My Drive/NT204/dataset/'
# Đường dẫn dành cho kaggle
DATASET_PATH = '/kaggle/working/'
TRAINSET_PATH = DATASET_PATH + 'train_set.csv'
TESTSET_PATH = DATASET_PATH + 'test_set.csv'

# Tạo thư mục để tải dataset
if not os.path.exists(DATASET_PATH):
  os.makedirs(DATASET_PATH)

'''
Tải tập train (train_set.csv) và tập test (test_set.csv) vào thư mục /content/drive/My Drive/NT204/dataset/ trên drive.
Lưu ý: Đặt đúng tên các file.
- Tải các tập dữ liệu tại: https://drive.google.com/drive/folders/1nYyDqbmichotvI9eSOrKo-yRwtLUy_iM?usp=sharing
'''

'\nTải tập train (train_set.csv) và tập test (test_set.csv) vào thư mục /content/drive/My Drive/NT204/dataset/ trên drive.\nLưu ý: Đặt đúng tên các file.\n- Tải các tập dữ liệu tại: https://drive.google.com/drive/folders/1nYyDqbmichotvI9eSOrKo-yRwtLUy_iM?usp=sharing\n'

In [ ]:
# Tiền xử lý dữ liệu
lable_encoder = preprocessing.LabelEncoder()

# Tập train
data_train = pd.read_csv(TRAINSET_PATH)

x_train, y_train = data_train.iloc[:, :-1], data_train.iloc[:, [-1]]
y_train = y_train.apply(lable_encoder.fit_transform)
x_train, y_train = x_train.to_numpy(), y_train.to_numpy()

# Lấy số thuộc tính và số lớp
NUM_FEATURES = x_train.shape[1] # Số thuộc tính bằng số chiều (số cột của x_train hoặc x_test)
NUM_CLASSES = np.unique(y_train).size # Số class bằng tổng số các giá trị khác nhau của y_train

# one-hot encoding label
y_train = tf.keras.utils.to_categorical(y_train)

In [ ]:
print("Number of features: %s" %NUM_FEATURES)
print("Number of classes: %s" %NUM_CLASSES)

Number of features: 42
Number of classes: 11


In [ ]:
# Xem chi tiết các cột trong dataset
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1778032 entries, 0 to 1778031
Data columns (total 43 columns):
 #   Column            Dtype  
---  ------            -----  
 0   dsport            float64
 1   proto             float64
 2   state             float64
 3   dur               float64
 4   sbytes            float64
 5   dbytes            float64
 6   sttl              float64
 7   dttl              float64
 8   sloss             float64
 9   dloss             float64
 10  service           float64
 11  Sload             float64
 12  Dload             float64
 13  Spkts             float64
 14  Dpkts             float64
 15  swin              float64
 16  dwin              float64
 17  stcpb             float64
 18  dtcpb             float64
 19  smeansz           float64
 20  dmeansz           float64
 21  trans_depth       float64
 22  res_bdy_len       float64
 23  Sjit              float64
 24  Djit              float64
 25  Sintpkt           float64
 26  Dintpkt       

In [ ]:
# xem số lượng mẫu của từng class
data_train['Label'].value_counts()

Label
Normal            1553134
Generic            150837
Exploits            31167
Fuzzers             16972
DoS                 11447
Reconnaissance       9791
Analysis             1874
Backdoor             1256
Shellcode            1058
Backdoors             374
Worms                 122
Name: count, dtype: int64

In [ ]:
# giảm số lượng mẫu thuộc 2 lớp cao nhất
data_train.drop(data_train[data_train['Label'] == 'Normal'].sample(frac=0.9).index, inplace=True)
data_train.drop(data_train[data_train['Label'] == 'Generic'].sample(frac=0.8).index, inplace=True)
data_train['Label'].value_counts()

Label
Normal            155313
Exploits           31167
Generic            30167
Fuzzers            16972
DoS                11447
Reconnaissance      9791
Analysis            1874
Backdoor            1256
Shellcode           1058
Backdoors            374
Worms                122
Name: count, dtype: int64

## Tạo và train model

In [ ]:
# Tạo mô hình deep learning (CNN)
model = tf.keras.Sequential(
  [
      # 1 conv_block bao gồm 2 lớp convolutional với kernel size là 1,  2 lớp có kernel size là 5
      # 1 lớp Layer Normalization và 1 lớp Max Pooling
      tf.keras.layers.Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(NUM_FEATURES, 1), padding='same'),
      tf.keras.layers.Conv1D(filters=64, kernel_size=5, activation='relu', padding='same'),
      tf.keras.layers.Conv1D(filters=64, kernel_size=1, activation='relu', padding='same'),
      tf.keras.layers.Conv1D(filters=64, kernel_size=5, activation='relu'),
      tf.keras.layers.LayerNormalization(),
      tf.keras.layers.MaxPooling1D(pool_size=(2), strides=2),

      # Tăng số lượng filter sau 1 block
      tf.keras.layers.Conv1D(filters=128, kernel_size=1, activation='relu', padding='same'),
      tf.keras.layers.Conv1D(filters=128, kernel_size=5, activation='relu', padding='same'),
      tf.keras.layers.Conv1D(filters=128, kernel_size=1, activation='relu', padding='same'),
      tf.keras.layers.Conv1D(filters=128, kernel_size=5, activation='relu'),
      tf.keras.layers.LayerNormalization(),
      tf.keras.layers.MaxPooling1D(pool_size=(2), strides=2),


      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(256, activation='relu'),
      tf.keras.layers.Dense(256, activation='relu'),
      tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

# Compile mô hình
model.compile(optimizer='adam', loss = tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Xem thông tin của mô hình
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 42, 64)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 42, 64)         │        20,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 42, 64)         │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 38, 64)         │        20,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization             │ (None, 38, 64)         │           128 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 19, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 19, 128)        │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 19, 128)        │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, 19, 128)        │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ (None, 15, 128)        │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_1           │ (None, 15, 128)        │           256 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 7, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 896)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       229,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 11)             │         2,827 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 532,939 (2.03 MB)

 Trainable params: 532,939 (2.03 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Huấn luyện mô hình
model.fit(x=x_train, y=y_train, batch_size=1024, epochs=20, shuffle=True)

Epoch 1/20
   6/1737 ━━━━━━━━━━━━━━━━━━━━ 41s 24ms/step - accuracy: 0.5351 - loss: 1.7881     

I0000 00:00:1717859228.298111    4799 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1717859228.319921    4799 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1737/1737 ━━━━━━━━━━━━━━━━━━━━ 67s 28ms/step - accuracy: 0.9654 - loss: 0.1009
Epoch 2/20
   1/1737 ━━━━━━━━━━━━━━━━━━━━ 3:31 122ms/step - accuracy: 0.9834 - loss: 0.0507

W0000 00:00:1717859276.979348    4802 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1737/1737 ━━━━━━━━━━━━━━━━━━━━ 41s 23ms/step - accuracy: 0.9759 - loss: 0.0609
Epoch 3/20
1737/1737 ━━━━━━━━━━━━━━━━━━━━ 41s 24ms/step - accuracy: 0.9770 - loss: 0.0583
Epoch 4/20
1737/1737 ━━━━━━━━━━━━━━━━━━━━ 41s 23ms/step - accuracy: 0.9778 - loss: 0.0555
Epoch 5/20
1737/1737 ━━━━━━━━━━━━━━━━━━━━ 41s 23ms/step - accuracy: 0.9781 - loss: 0.0542
Epoch 6/20
1737/1737 ━━━━━━━━━━━━━━━━━━━━ 41s 23ms/step - accuracy: 0.9784 - loss: 0.0529
Epoch 7/20
1737/1737 ━━━━━━━━━━━━━━━━━━━━ 41s 23ms/step - accuracy: 0.9789 - loss: 0.0517
Epoch 8/20
1737/1737 ━━━━━━━━━━━━━━━━━━━━ 41s 23ms/step - accuracy: 0.9790 - loss: 0.0514
Epoch 9/20
1737/1737 ━━━━━━━━━━━━━━━━━━━━ 41s 23ms/step - accuracy: 0.9797 - loss: 0.0498
Epoch 10/20
1737/1737 ━━━━━━━━━━━━━━━━━━━━ 41s 23ms/step - accuracy: 0.9795 - loss: 0.0498
Epoch 11/20
1737/1737 ━━━━━━━━━━━━━━━━━━━━ 40s 23ms/step - accuracy: 0.9799 - loss: 0.0490
Epoch 12/20
1737/1737 ━━━━━━━━━━━━━━━━━━━━ 40s 23ms/step - accuracy: 0.9802 - loss: 0.0480
Epoch 13/20
1737/1

## Kết quả

In [ ]:
# Lưu mô hình trên kaggle
model.save('/kaggle/working/model.h5')
# Lưu mô hình trên google drive
# model.save('/content/drive/My Drive/NT204/')

In [ ]:
# Load dữ liệu để kiểm tra mô hình
data_test = pd.read_csv(TESTSET_PATH)
x_test, y_test = data_test.iloc[:, :-1], data_test.iloc[:, [-1]]
y_test = y_test.apply(lable_encoder.fit_transform)
x_test, y_test = x_test.to_numpy(), y_test.to_numpy()

In [ ]:
# Kiểm tra mô hình với tập test
preds = model.predict(x_test, batch_size=1024)

 28/745 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step

W0000 00:00:1717860049.188802    4800 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


745/745 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step


W0000 00:00:1717860053.430480    4799 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [ ]:
# Tính toán kết quả
y_pred = np.argmax(preds, axis=1)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

print('Accuracy: %s\nPrecision: %s\nRecal: %s\nF1-Score: %s' %(accuracy, precision, recall, f1))

Accuracy: 0.9803389697053208
Precision: 0.9797123304303778
Recal: 0.9803389697053208
F1-Score: 0.9790439984348012
